In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('train.csv')

In [3]:

df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:

###Drop Nan Values
df=df.dropna()

In [5]:

## Get the Independent Features

X=df.drop('label',axis=1)

In [6]:
## Get the Dependent features
y=df['label']

In [7]:

y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [8]:
X.shape

(18285, 4)

In [9]:
import tensorflow as tf

In [10]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [11]:
### Vocabulary size
voc_size=5000

In [12]:

messages=X.copy()

In [13]:

messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [14]:

messages.reset_index(inplace=True)

In [15]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [16]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[4553, 943, 2471, 4188, 4430, 2225, 4502, 184, 4520, 2850],
 [4110, 2134, 2608, 2939, 1443, 2414, 2153],
 [759, 2254, 2133, 2927],
 [2951, 417, 1572, 4441, 4367, 2301],
 [4637, 1443, 357, 3066, 938, 1796, 1443, 2149, 814, 613],
 [4448,
  2301,
  2134,
  2599,
  140,
  2667,
  4619,
  1333,
  4201,
  4142,
  1354,
  4089,
  353,
  2905,
  2153],
 [1880, 187, 1208, 4501, 1954, 3209, 1223, 723, 1109, 2561, 1174],
 [1125, 4785, 4619, 1959, 184, 4552, 2667, 2053, 1109, 2561, 1174],
 [2352, 4629, 4045, 2928, 3593, 1176, 4425, 2905, 2667, 4678],
 [765, 2287, 4070, 3680, 238, 1415, 262, 21],
 [3830, 366, 2732, 707, 4106, 4477, 19, 4569, 1932, 2788, 4568],
 [4441, 4773, 4430, 1176, 2667, 184],
 [2171, 995, 1444, 400, 1416, 2976, 4618, 4209, 1731],
 [3238, 4693, 1320, 2202, 4781, 1094, 1014, 1109, 2561, 1174],
 [4971, 4687, 1648, 1904, 1963, 1109, 2561, 1174],
 [443, 1551, 2941, 3204, 3902, 2219, 2131, 1871, 1448, 1552],
 [4660, 4629, 2134],
 [544, 2621, 2860, 4872, 2667, 3241, 4891, 2153],
 [3

In [18]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...  184 4520 2850]
 [   0    0    0 ... 1443 2414 2153]
 [   0    0    0 ... 2254 2133 2927]
 ...
 [   0    0    0 ... 1109 2561 1174]
 [   0    0    0 ... 1225 1526 3852]
 [   0    0    0 ... 1821 4349  732]]


In [19]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 4553,
        943, 2471, 4188, 4430, 2225, 4502,  184, 4520, 2850], dtype=int32)

In [38]:
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 20, 40)            200000    
                                                                 
 lstm_1 (LSTM)               (None, 100)               56400     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [20]:
## Creating model
embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 bidirectional (Bidirection  (None, 200)               112800    
 al)                                                             
                                                                 
 dropout (Dropout)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 313001 (1.19 MB)
Trainable params: 313001 (1.19 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [21]:

len(embedded_docs),y.shape

(18285, (18285,))

In [22]:
 
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [23]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [39]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=8,batch_size=100)

Epoch 1/8
123/123 [==============================] - 6s 33ms/step - loss: 0.4015 - accuracy: 0.8085 - val_loss: 0.2228 - val_accuracy: 0.9107
Epoch 2/8
123/123 [==============================] - 4s 31ms/step - loss: 0.1494 - accuracy: 0.9398 - val_loss: 0.2116 - val_accuracy: 0.9188
Epoch 3/8
123/123 [==============================] - 4s 30ms/step - loss: 0.1024 - accuracy: 0.9621 - val_loss: 0.2326 - val_accuracy: 0.9157
Epoch 4/8
123/123 [==============================] - 4s 30ms/step - loss: 0.0762 - accuracy: 0.9731 - val_loss: 0.2599 - val_accuracy: 0.9077
Epoch 5/8
123/123 [==============================] - 3s 28ms/step - loss: 0.0504 - accuracy: 0.9832 - val_loss: 0.3005 - val_accuracy: 0.9079
Epoch 6/8
123/123 [==============================] - 4s 32ms/step - loss: 0.0337 - accuracy: 0.9895 - val_loss: 0.3750 - val_accuracy: 0.9100
Epoch 7/8
123/123 [==============================] - 4s 33ms/step - loss: 0.0211 - accuracy: 0.9936 - val_loss: 0.3480 - val_accuracy: 0.9039
Epoch 

In [40]:
predictions = model.predict(X_test)
y_pred = (predictions.flatten() >= 0.5).astype(int)

189/189 [==============================] - 1s 4ms/step


In [34]:

from sklearn.metrics import confusion_matrix

In [41]:
confusion_matrix(y_test,y_pred)

array([[3114,  305],
       [ 248, 2368]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9083678541839271

In [43]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.93      0.91      0.92      3419
           1       0.89      0.91      0.90      2616

    accuracy                           0.91      6035
   macro avg       0.91      0.91      0.91      6035
weighted avg       0.91      0.91      0.91      6035

